In [ ]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         FindResourceOrThrow, IrisInConfigurationSpace,
                         IrisOptions, JointSliders, LoadModelDirectives,
                         MeshcatVisualizer, Parser, ProcessModelDirectives,
                         RigidTransform, RollPitchYaw, StartMeshcat)

from reproduction.util import *


In [ ]:
meshcat = StartMeshcat()

In [ ]:

seed_points = {
    'nominal':
    [1.2, -0.7, 1.2, -2.1, -1.8, -0.1, -1.3, -2.5, -1.1, -1.3, 1.6, 0]
}

def parse_homecart(plant):
    parser = Parser(plant)
    parser.package_map().Add("gcs", GcsDir())
    directives = LoadModelDirectives(
        FindModelFile('models/homecart.yaml'))
    ProcessModelDirectives(directives, plant, parser)

def parse_sugarbox_scenario(plant):
    parse_homecart(plant)
    Parser(plant).AddModelFromFile(
        FindResourceOrThrow(
            'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    X_sugar = RigidTransform(RollPitchYaw(-np.pi/2, 0, 0), [0.36, 0.36, 0.11])
    plant.WeldFrames(plant.world_frame(),
                     plant.GetFrameByName('base_link_sugar'), X_sugar)


In [ ]:
def teleop_seeds(q0=None):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_sugarbox_scenario(plant)
#    parse_homecart(plant)
#    Parser(plant).AddModelFromFile(
#        FindResourceOrThrow(
#            'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    sliders = builder.AddSystem(JointSliders(meshcat, plant, initial_value=q0))
    diagram = builder.Build()
    sliders.Run(diagram)
    # Print current value
    context = sliders.CreateDefaultContext()
    q = sliders.get_output_port().Eval(context)
    meshcat.DeleteAddedControls()
    print(f"q = {q}")
    return q

teleop_seeds(seed_points['nominal'])
#teleop_seeds()

In [ ]:
def ik_seeds(seed_points['nominal']):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    plant.Finalize()



In [ ]:
def run_iris(seed):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    plant.Finalize()
    diagram = builder.Build()

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    
    iris_options = IrisOptions()
    iris_options.require_sample_point_is_contained = True
    iris_options.iteration_limit = 5
    iris_options.termination_threshold = -1
    iris_options.relative_termination_threshold = 0.02
    iris_options.enable_ibex = False

    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Time:", np.round((time.time() - start_time)/60., 4),
          "minutes.\tFaces", len(hpoly.b()), flush=True)
    return hpoly

run_iris(seed_points['nominal'])